# Get jingle audiofiles

## Prepare environment

In [ ]:
!pip install -q podcast-api inaSpeechSegmenter transliterate pickle5

     |████████████████████████████████| 389kB 8.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 13.9MB 211kB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 92kB 12.0MB/s 
     |████████████████████████████████| 133kB 55.2MB/s 


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Coursework

/content/drive/MyDrive/Coursework


In [ ]:
import re
import os
import pickle5 as pickle
import pandas as pd

from tqdm.notebook import tqdm
from transliterate import translit
from listennotes import podcast_api
from credentials import listennotes_key
from inaSpeechSegmenter import Segmenter


client = podcast_api.Client(api_key=listennotes_key)
seg = Segmenter()

## Perform jingle extraction

In [ ]:
with open("jingles_df.pkl", "rb") as inp:
  jingles_df = pickle.load(inp)
jselect_df = jingles_df.loc[jingles_df["j_status"].isin(["+", "2", "!!", 2])].reset_index(drop=True)
jselect_df

,j_status,title,category_ln,category_yp,ln_id
0,+,Задвижки по недвижке,Новости,Новости,a37edc43456b4fa5ae5c7839040dc01e
1,+,Podcasts Digest,Новости,Новости,2bc20b9838734d8e819b75eef2b443c1
2,+,Давай голосом,Новости,Новости,51b771cd98ad4e759f0b38a00441ee5b
3,+,Первым делом,Новости,Новости,d0900de5c87f4197b676c68ec9a6048b
4,+,Не верю!,Новости,Искусство,bbda31b8cf20451cae83fcc5eb4fb1ae
...,...,...,...,...,...
182,+,Сперва роди,Для всей семьи,Для всей семьи,1ce42458022f44e7b2849584560e4311
183,+,Три на Три. Подкаст,Юмор,Юмор,12d704bc20c049229cda59630ce4c4a4
184,+,Важный вопрос,Юмор,Юмор,f6099fb264e94aa1ba3f8e02d1f0f758
185,+,Хочу не могу 18+,Юмор,Общество и культура,3be59b3b87ec46b9a0979d8b2c0f0075


In [ ]:
#!mkdir jingles

pqueue = tuple(zip(list(jselect_df["title"]), list(jselect_df["ln_id"]), list(jselect_df["j_status"])))

for title, pid, method in tqdm(pqueue):
  method = str(method)

  response = client.fetch_podcast_by_id(
    id = pid,
    sort = 'recent_first'
  )

  mp3_link = response.json()["episodes"][1 if method == "2" else 0]["audio"]
  !wget -q {mp3_link} -O {pid}
  !ffmpeg -ss 0 -i {pid} -t 180 -c:a libmp3lame {pid}_trimmed.mp3 1>/dev/null 2>&1

  segmentation = seg(pid + "_trimmed.mp3")

  _cnt = 2 if method == "!!" else 1

  cnt = 0
  start, end = 0, 0
  for e in segmentation:
    if e[0] == "music":
      cnt += 1
      if cnt == _cnt:
        start, end = e[1], e[2]
        break

  if not end:
    !rm {pid} {pid}_trimmed.mp3
    continue
  
  titler = re.sub(r"[^\w0-9_]", "", re.sub(r"\s", "_", translit(re.sub(r"[^\w\s]", "", title.lower()), "ru", reversed=True)))
  jname = titler + "_jingle.mp3"
  !ffmpeg -ss {str(start)} -i {pid}_trimmed.mp3 -t {str(end-start)} -c copy {jname} 1>/dev/null 2>&1

  !rm {pid} {pid}_trimmed.mp3
  !mv {jname} jingles

/usr/local/lib/python3.7/dist-packages/pyannote/algorithms/utils/viterbi.py:88: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for e, c in six.moves.zip(emission.T, consecutive)
/usr/local/lib/python3.7/dist-packages/pyannote/algorithms/utils/viterbi.py:97: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for e, c in six.moves.zip(constraint.T, consecutive)
/usr/local/lib/python3.7/dist-packages/inaSpeechSegmenter/segmenter.py:60: RuntimeWarning: invalid value encountered in subtract
  data = (data - np.mean(data, axis=1).reshape((len(data), 1))) / np.std(data, axis=1).reshape((len(data), 1))
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:20

In [ ]:
jselect_df["jingle_fname"] = [re.sub(r"[^\w0-9_]", "", re.sub(r"\s", "_", translit(re.sub(r"[^\w\s]", "", t.lower()), "ru", reversed=True))) + "_jingle.mp3" \
                              for t in list(jselect_df["title"])]
jselect_df = jselect_df.loc[[fname in os.listdir("jingles") for fname in list(jselect_df["jingle_fname"])]].reset_index(drop=True)
jselect_df.to_pickle("jselect_df.pkl")
jselect_df

,j_status,title,category_ln,category_yp,ln_id,jingle_fname
0,+,Задвижки по недвижке,Новости,Новости,a37edc43456b4fa5ae5c7839040dc01e,zadvizhki_po_nedvizhke_jingle.mp3
1,+,Podcasts Digest,Новости,Новости,2bc20b9838734d8e819b75eef2b443c1,podcasts_digest_jingle.mp3
2,+,Давай голосом,Новости,Новости,51b771cd98ad4e759f0b38a00441ee5b,davaj_golosom_jingle.mp3
3,+,Первым делом,Новости,Новости,d0900de5c87f4197b676c68ec9a6048b,pervym_delom_jingle.mp3
4,+,Не верю!,Новости,Искусство,bbda31b8cf20451cae83fcc5eb4fb1ae,ne_verju_jingle.mp3
...,...,...,...,...,...,...
171,+,Сперва роди,Для всей семьи,Для всей семьи,1ce42458022f44e7b2849584560e4311,sperva_rodi_jingle.mp3
172,+,Три на Три. Подкаст,Юмор,Юмор,12d704bc20c049229cda59630ce4c4a4,tri_na_tri_podkast_jingle.mp3
173,+,Важный вопрос,Юмор,Юмор,f6099fb264e94aa1ba3f8e02d1f0f758,vazhnyj_vopros_jingle.mp3
174,+,Хочу не могу 18+,Юмор,Общество и культура,3be59b3b87ec46b9a0979d8b2c0f0075,hochu_ne_mogu_18_jingle.mp3


## Post-cleanup

Manual analysis performed here

In [ ]:
jingles_final = pd.read_excel("jingles_final.xlsx")
jingles_final = jingles_final[[not(b) for b in list(jingles_final["exclude"])]]
jingles_final = jingles_final.sort_values(by="category_yp").reset_index(drop=True)
jingles_final = jingles_final[["title", "category_yp", "jingle_fname"]]
jingles_final.columns = ["title", "category", "jingle_fname"]
jingles_final.to_excel("podcasts_list.xlsx")
jingles_final

,title,category,jingle_fname
0,Маркетинговая рубка,Бизнес и работа,marketingovaja_rubka_jingle.mp3
1,Богема и Маркетинг,Бизнес и работа,bogema_i_marketing_jingle.mp3
2,Взяла и сделала,Бизнес и работа,vzjala_i_sdelala_jingle.mp3
3,Нормальные деньги,Бизнес и работа,normalnye_dengi_jingle.mp3
4,ПЛЮС-МИНУС ДЕНЬГИ,Бизнес и работа,pljusminus_dengi_jingle.mp3
...,...,...,...
86,Вне игры,Спорт,vne_igry_jingle.mp3
87,Наш Чемп,Спорт,nash_chemp_jingle.mp3
88,Дешифратор,Спорт,deshifrator_jingle.mp3
89,Футбольный механизм,Спорт,futbolnyj_mehanizm_jingle.mp3


Cut last 10% of every selected jingle to exclude narrative start

In [ ]:
!mv jingles all_jingles
!mkdir selected_jingles

for jname in tqdm(list(jingles_final["jingle_fname"])):
  jdur = !ffprobe -i all_jingles/{jname} -show_entries format=duration -v quiet -of csv="p=0"
  jdur = float(str(jdur[0]))
  !ffmpeg -ss 0 -i all_jingles/{jname} -t {str(jdur*0.9)} -ar 44100 -c:a pcm_s16le selected_jingles/{jname[:-4]+".wav"} 1>/dev/null 2>&1